In [ ]:
from transformers import AutoTokenizer

In [ ]:
model_path = "./models/Qwen3-4B-Thinking-2507"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
with open("qwen3_configs/chat_template.j2") as f:
    chat_template = f.read()
tokenizer.chat_template = chat_template

In [ ]:
from datasets import load_from_disk

ds = load_from_disk("data/grids_v15/arc2_evaluation6")
ds

In [ ]:
def get_token_length(sample: dict):
    text = tokenizer.apply_chat_template(
        sample["messages"],
        tokenize=False,
    )
    tokens = tokenizer.encode(text + "<|endoftext|>")
    sample["tokens"] = tokens
    return sample

ds = ds.map(get_token_length, num_proc=8)
ds

In [ ]:
all_tokens = []
for sample in ds:
    all_tokens.extend(sample["tokens"])
all_tokens = set(all_tokens)
len(all_tokens)

In [ ]:
mapping = []
new_vocab = dict()
for token in sorted(all_tokens):
    v = tokenizer.convert_ids_to_tokens(token)
    new_vocab[v] = len(mapping)
    print(token, v)
    mapping.append(token)
len(mapping)

In [ ]:
repr(new_vocab)

In [ ]:
from safetensors import safe_open

path_file = f"{model_path}/model-00001-of-00003.safetensors"

tensors = {}
with safe_open(path_file, framework="pt", device="cpu") as f:
   for key in f.keys():
       tensors[key] = f.get_tensor(key)

embed_tokens_weight = tensors["model.embed_tokens.weight"]
# embed_tokens_weight = tensors["lm_head.weight"]

print(embed_tokens_weight.shape, embed_tokens_weight.dtype)

In [ ]:
import torch

padded_mapping = mapping #+ [151643] * 20

row_select = torch.tensor(padded_mapping)

selected_embedding = torch.index_select(embed_tokens_weight, 0, row_select)
print(selected_embedding.shape, selected_embedding.dtype)

In [ ]:
from safetensors.torch import save_file

tensors["model.embed_tokens.weight"] = selected_embedding
# tensors["lm_head.weight"] = selected_embedding.clone()

output_path = model_path + "-16t"

save_file(tensors, f"{output_path}/model-00001-of-00003.safetensors")